<img src="./img/sktime-logo-text-horizontal.jpg" alt="sktime logo" style="width: 50%; max-width: 600px;">  &
<img src="./img/NR-logo_utvidet_stor_rgb.gif" alt="NR logo" style="width: 30%; max-width: 200px;"> 

### Agenda

1. Detection tasks
2. Detector -- conceptual model
3. Change detection

    * Interface
    * Composable algorithm framework
    * Under the hood: costs and change scores

4. Segment anomaly detection

    * Interface
    * Subset anomaly detection
    * Under the hood: savings
5. Example: HVAC system data
6. Example: Change in the covariance matrix (if time)

# The detection module
<!-- <img src="img/annotation_tree.png" width="800"> -->

Experimental module, still under heavy development.

Some discrepancies between `sktime` and `skchange` are still expected for some time.

Contributions appreciated!

# Detection tasks

1. Change detection
2. Segmentation
3. Point anomaly/outlier detection
4. Segment anomaly detection (special case of segmentation)


## Change detection

Detect points in time where the data distribution changes.

In [1]:
from skchange.datasets import generate_changing_data
from utils import plot_changepoint_illustration


n = 310
cpts = [50, 150]
means = [0.0, -3, 0.0]
df_2cpts = generate_changing_data(n, cpts, means, random_state=3)
plot_changepoint_illustration(df_2cpts, cpts)

## Segmentation

Divide the time series into segments based on certain critera.
* Change detection on interval format + labels for the segments.
* The labels can be used to multiple disconnected segments.
* Change detection = segmentation with unique labels.

In [2]:
import pandas as pd

from utils import plot_segmentation_illustration

extended_cpts = [0] + cpts + [n]
segments = pd.DataFrame({
    "ilocs":
    [
        pd.Interval(extended_cpts[i], extended_cpts[i + 1], closed="left")
        for i in range(len(extended_cpts) - 1)
    ],
    "labels": [0, 1, 0]
})
plot_segmentation_illustration(df_2cpts, segments)

In [10]:
segments

,ilocs,labels
0,"[0, 50)",0
1,"[50, 150)",1
2,"[150, 310)",0


### Use cases of change detection and segmentation

* Data cleaning: Remove segments that are not relevant for the analysis.
* Preprocessing: Divide the data into homogenous parts for individual analysis.
* Detect interesting patterns: Anomaly detection, motif discovery, state transitions.

## Point anomaly detection

Detect individual data points that are different from the rest of the data.

In [15]:
import numpy as np

from utils import plot_point_anomaly_illustration

df_3outliers = pd.DataFrame(np.random.normal(0, 1, n))
outliers = [60, 238, 290]
df_3outliers.iloc[outliers] = np.random.uniform(
    4, 8, (len(outliers), df_3outliers.shape[1])
)
plot_point_anomaly_illustration(df_3outliers, outliers)

## Segment anomaly detection

Detect data segments that are different from the rest of the data.

* A special case of segmentation where the data not in any segments (the normal data) are implicitly given the same label.
<!-- 
Can be viewed as a special case of segmentation and change detection:

* Segment anomaly = A change away from the baseline data behaviour + a change back again. -->

In [16]:
from utils import plot_segment_anomaly_illustration

anomaly = segments.iloc[1:2]
plot_segment_anomaly_illustration(df_2cpts, anomaly)

### Use cases of anomaly detection

* Data cleaning: Remove anomalies from the data.
* Detect interesting events: Fault detection, fraud detection, environmental monitoring, health monitoring etc.

# The Detector -- conceptual model
What does all the problems above have in common?

1. Input: A time series.
2. Output: Locations of events in the time series.
    * Changepoints,
    * Segments,
    * Point anomalies.
    * Segment anomalies
    * ...

Length(output) = Number of detected events.

All detectors are built around this model.

# Change detection

* Let us look at change detection in more detail.
* We will use this task to introduce the general detector interface and concepts.

Let us generate some toy data that will be used throughout the tutorial.

To not oversimplify the illustration, we will use a 3-dimensional time series with 2 anomalous segments.

In [21]:
from skchange.datasets import generate_anomalous_data

# Generate data
n = 300
anomalies = [
    (100, 120),
    (250, 300),
]
means = [
    np.array([8.0, 0.0, 0.0]),
    np.array([2.0, 3.0, 5.0]),
]
df_3d = generate_anomalous_data(n, anomalies=anomalies, means=means, random_state=3)
df_3d.columns = ["var0", "var1", "var2"]
df_3d.index.name = "time"
df_3d

,var0,var1,var2
time,,,
0,1.788628,0.436510,0.096497
1,-1.863493,-0.277388,-0.354759
2,-0.082741,-0.627001,-0.043818
3,-0.477218,-1.313865,0.884622
4,0.881318,1.709573,0.050034
...,...,...,...
295,1.181978,3.571703,6.375051
296,2.403536,2.262568,4.251524
297,3.365367,2.201864,5.089317


In [22]:
from utils import plot_multivariate_time_series

fig_3d = plot_multivariate_time_series(df_3d)
fig_3d

## The `MovingWindow` detector

The `MovingWindow` serve as our dummy detector.

Fastest and conceptually simplest search method available in `skchange`:

1. Move a window with `bandwidth` samples to the left and right of a split point across the time series.
2. For each split, calculate a change score: A measure of the difference between the two intervals.
3. Detect a change if the change score exceeds a threshold.

It has three primary parameters:

1. `change_score`: An `skchange` object that measure the difference between two data intervals.
2. `bandwidth`: The number of data points on either side of the split in the window.
3. `threshold_scale`: Scaling factor for the default threshold. The default threshold depends on the number of samples, the number of variables and `bandwidth`.

## Interface

### Initialize
Set the hyperparameters.

In [ ]:
from skchange.change_detectors import MovingWindow
from skchange.change_scores import CUSUM

detector = MovingWindow(
    change_score=CUSUM(),  # measures the difference in mean between two intervals.
    bandwidth=20,
)
detector

MovingWindow(bandwidth=20, change_score=CUSUM())

### Fit
Fit the detector to training data.

For the `MovingWindow` detector:
* Set the `threshold` based on the shape of the training data.

In [25]:
df_3d

,var0,var1,var2
time,,,
0,1.788628,0.436510,0.096497
1,-1.863493,-0.277388,-0.354759
2,-0.082741,-0.627001,-0.043818
3,-0.477218,-1.313865,0.884622
4,0.881318,1.709573,0.050034
...,...,...,...
295,1.181978,3.571703,6.375051
296,2.403536,2.262568,4.251524
297,3.365367,2.201864,5.089317


In [26]:
detector = detector.fit(df_3d)
detector

MovingWindow(bandwidth=20, change_score=CUSUM())

Supervised detectors are also supported:
`fit(self, X, y=true_labels)`

### Predict

* Detect events on new data.
* Sparse format: One entry per detected event.

In [27]:
detections = detector.predict(df_3d)
detections

,ilocs
0,100
1,120
2,250


In [28]:
from utils import add_changepoint_vlines

pred_output_fig = plot_multivariate_time_series(df_3d)
add_changepoint_vlines(pred_output_fig, detections)

### Transform
* Detect events on new data.
* Dense format: Labels each row of the input time series.

Note: Currently you get the segment labels from `transform` of change detectors in `skchange`. In `sktime`, you get an indicator for the change point locations. To be aligned.

In [29]:
labels = detector.transform(df_3d)
labels

,labels
time,
0,0
1,0
2,0
3,0
4,0
...,...
295,3
296,3
297,3


In [30]:
import plotly.express as px

px.line(labels)

<!-- ### Transform scores (optional)
Return detection scores for each row of the input time series. -->

In [31]:
detection_scores = detector.transform_scores(df_3d)
detection_scores

time
0     -22.81513
1     -22.81513
2     -22.81513
3     -22.81513
4     -22.81513
         ...   
295   -22.81513
296   -22.81513
297   -22.81513
298   -22.81513
299   -22.81513
Name: score, Length: 300, dtype: float64

In [34]:
detector.get_params()

{'bandwidth': 20,
 'change_score': CUSUM(),
 'min_detection_interval': 1,
 'penalty': None}

In [37]:
import plotly.express as px

scores_fig = px.line(detection_scores)
scores_fig

The peaks above the threshold correspond to the changepoints detected by the moving window algorithm.

## Other change detectors

We can easily swap the `MovingWindow` detector with other detectors in `skchange`.

In [41]:
from skchange.change_detectors import PELT
from skchange.costs import L2Cost

detector = PELT(L2Cost())
detector.fit_predict(df_3d)

,ilocs
0,100
1,120
2,250


In [42]:
from skchange.change_detectors import SeededBinarySegmentation
from skchange.costs import L2Cost

detector = SeededBinarySegmentation(L2Cost())
detector.fit_predict(df_3d)

,ilocs
0,100
1,120
2,250


The content of `transform_scores` will depend on the detector.

## Composable algorithm framework

There are two large classes of change detection algorithms:

* Cost-based algorithms.
* Change score-based algorithms.

The `MovingWindow` detector is based on change scores.

* Measures the difference between two data intervals.

In [43]:
change_detector = MovingWindow(
    change_score=CUSUM(),  # change in mean
    bandwidth=20
)
change_detector.fit_predict(df_3d)

,ilocs
0,100
1,120
2,250


`PELT`, however, is based on costs.

* Measures the error/loss/cost of a model over a data interval.

In [44]:
change_detector = PELT(
    cost=L2Cost(),  # change in mean, squared error cost
)
change_detector.fit_predict(df_3d)

,ilocs
0,100
1,120
2,250


All detectors in `skchange` are composed of either a cost or a change/anomaly score.

* The cost/score component specify what data characteristics to look for changes in.
* They implement an interface that allows efficient evaluation over many data intervals and/or splits.
* `numba` is used to for speed-ups where possible.


This is a key feature of `skchange`. It means:

**To get a new detector for finding changes or anomalies in any data characteristic of interest, all you need to do is to find an existing or implement a new cost or change score class.**

Note: Similar pattern as in the `ruptures` package, but more general and performance oriented.


In [45]:
from skchange.costs import GaussianCost

change_detector = PELT(
    cost=GaussianCost(),  # change in variance
)
change_detector.fit_predict(df_3d)

,ilocs
0,100
1,120
2,250


In [46]:
from skchange.costs import MultivariateGaussianCost

change_detector = PELT(
    cost=MultivariateGaussianCost(),  # change in covariance matrix
    min_segment_length=4,
)
change_detector.fit_predict(df_3d)

,ilocs
0,100
1,120
2,250


### Costs -> Change scores
Moreover, all costs can be used to construct a change score.
* => Change score-based algorithms also accept costs as input.
* Costs are converted to change scores internally.

(The reverse is not true: All change scores cannot be reduced to costs.)

In [48]:
from skchange.costs import L2Cost

change_detector = MovingWindow(
    change_score=L2Cost(),  # change in mean.
    bandwidth=20
    )
change_detector.fit_predict(df_3d)

,ilocs
0,100
1,120
2,250


In [50]:
change_detector = MovingWindow(
    change_score=GaussianCost(),  # change in variance.
    bandwidth=20,
)
change_detector.fit_predict(df_3d)

,ilocs
0,100
1,120
2,250


In [51]:
from skchange.costs import MultivariateGaussianCost

change_detector = MovingWindow(
    change_score=MultivariateGaussianCost(),  # Change in covariance matrix.
    bandwidth=20,
)
change_detector.fit_predict(df_3d)

,ilocs
0,100
1,120
2,250


The consequence of this is huge:

**Once a cost has been implemented, it can be used by ALL detectors in `skchange`.**

Implementing change scores directly can be reserved for cases where it cannot be reduced to a cost, or where there are big computational gains in computing it directly rather than through costs.

### Future development
Implement a lot of different costs.

Contributions welcome!

## Under the hood: Costs and change scores

Purpose of this section:
* Explain what happens under the hood in `skchange`.
* Not primarily meant to be used directly by the user.

### Example 1: L2 cost

* Model per interval: A constant mean.
* Output per interval: The squared error for a constant mean model.

Generate some Gaussian toy data with a single changepoint.

In [53]:
from skchange.datasets import generate_alternating_data

change_point = 50
single_cpt_df = generate_alternating_data(
    n_segments=2, segment_length=change_point, mean=5, random_state=0
)
single_cpt_df

,var0
0,1.764052
1,0.400157
2,0.978738
3,2.240893
4,1.867558
...,...
95,5.706573
96,5.010500
97,6.785870
98,5.126912


In [54]:
single_cpt_fig = px.line(single_cpt_df)
single_cpt_fig

In [55]:
from skchange.costs import L2Cost

from utils import plot_interval_costs

cost = L2Cost()  # L2 cost function for a constant mean.
cost.fit(
    single_cpt_df
)  # Precomputes sums and sums of squares. Makes evaluation faster.
intervals = [[0, 30], [40, 60], [70, 85]]
cost_values = cost.evaluate(intervals)  # Uses precomputed sums + numba to evaluate.
plot_interval_costs(single_cpt_df, intervals, cost_values)

/Users/pabloparedes/Documents/darts_sktime_comparing/sktime-tutorial-pydata-global-2024/notebooks/utils.py:180: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



* The lower the cost, the better the fit. 
* High cost in the middle interval because a constant mean fits poorly due to the change point at index 50.

### Example 2: Change score based on L2 cost

* Evaluated over `(start, split, end)`.
* Compares `X[start:split]` and `X[split:end]`.

Constructing a change score from costs:
```
score.evaluate([start, split, end]) = cost.evaluate([start, end]) - (cost.evaluate([start, split]) + cost.evaluate([split, end]))
```
"score = cost of the interval without a change point - cost of the interval with a single change point"

In [56]:
from skchange.change_scores import ChangeScore
from skchange.costs import L2Cost

from utils import plot_interval_change_scores

change_score = ChangeScore(cost=L2Cost())
change_score.fit(single_cpt_df)
start_split_ends = [[0, 8, 30], [40, 50, 60], [70, 80, 85]]
change_score_values = change_score.evaluate(start_split_ends)
plot_interval_change_scores(single_cpt_df, start_split_ends, change_score_values)

The higher the score, the more evidence for a change. 

High score for interval 1 because the difference between the means before and after the split inside the interval is large.

# Segment anomaly detection
Now we will look at segment anomaly detection on the same 3-dimensional toy data as before.

In [57]:
plot_multivariate_time_series(df_3d)

## The `CAPA` detector

The `CAPA` (Collective And Point Anomalies) algorithm serve as our dummy anomaly detector. For simplicity, we will ignore the point anomaly capability.

It uses a type of anomaly score called a `saving` (more details later).
* The cost difference between a baseline parameter and an optimal parameter.
* The baseline parameter represents the normal data behaviour.
* A high saving indicates that the segment has a different behaviour than the baseline.
* The baseline parameter must be estimated in advance.
* As for change scores, savings can be constructed from costs.

Parameters:

* `collective_saving`: The saving to use for segment anomaly detection.
* `collective_penalty_scale`: The scaling factor of the penalty for adding a new segment anomaly.
* `min_segment_length`: The minimum length of a segment to consider.
* `max_segment_length`: The maximum length of a segment to consider.

Optimises the penalized saving by an efficient recursion.

## Interface

### Initialize
Set the hyperparameters.

In [60]:
from skchange.anomaly_detectors import CAPA
from skchange.anomaly_scores import L2Saving

detector = CAPA(
    segment_saving=L2Saving(),  # mean=0 vs mean=something else.
    min_segment_length=10,
    max_segment_length=100
)
detector

CAPA(max_segment_length=100, min_segment_length=10, segment_saving=L2Saving())

### Fit
Fit the detector to training data.

For the `CAPA` detector:
* Set the `collective_penalty` (and the `point_penalty`) based on the shape of the training data.

In [62]:
detector = detector.fit(df_3d)
detector

CAPA(max_segment_length=100, min_segment_length=10, segment_saving=L2Saving())

Supervised detectors are also supported:
`fit(self, X, y=true_labels)`

### Predict

* Detect events on new data.
* Sparse format: One entry per detected event.

In [63]:
detections = detector.predict(df_3d)
detections

,ilocs,labels
0,"[100, 120)",1
1,"[250, 300)",2


In [64]:
from utils import add_segmentation_vrects

pred_output_fig = plot_multivariate_time_series(df_3d)
add_segmentation_vrects(pred_output_fig, detections, colors=["red"])

### Transform
* Detect events on new data.
* Dense format: Labels each row of the input time series.

In [66]:
df_3d.index = range(df_3d.shape[0])
labels = detector.transform(df_3d)
labels

,labels
0,0
1,0
2,0
3,0
4,0
...,...
295,2
296,2
297,2
298,2


* Normal = label 0
* Segment anomalies = labels 1, 2, 3, ...

In [67]:
px.line(labels)

### Transform scores (optional)
Return detection scores for each row of the input time series.

In [68]:
detection_scores = detector.transform_scores(df_3d)
detection_scores

0         0.000000
1         0.000000
2         0.000000
3         0.000000
4         0.000000
          ...     
295    2943.564412
296    2971.524413
297    3011.451253
298    3059.507890
299    3101.575877
Name: score, Length: 300, dtype: float64

In [69]:
import plotly.express as px

px.line(detection_scores)

For `CAPA`, the scores are *the cumulative optimal savings*.

They increase when the saving is higher than the penalty.

## Subset anomaly detection

Identifies the variables that are affected by or cause the anomaly.

* `MVCAPA` (Multivariate Collective and Point Anomalies) is the only algorithm in `skchange` with this capability so far.

* It is a more complex version of `CAPA` that optimises the saving over all possible variables subsets.

In [70]:
from skchange.anomaly_detectors import MVCAPA

from utils import add_subset_segment_anomaly_vrects

subset_anomaly_detector = MVCAPA()
subset_anomalies = subset_anomaly_detector.fit_predict(df_3d)
print(subset_anomalies)
add_subset_segment_anomaly_vrects(fig_3d, subset_anomalies)

        ilocs  labels   icolumns
0  [100, 120)       1        [0]
1  [250, 300)       2  [2, 1, 0]


## Under the hood: Savings

Purpose of this section:
* Explain what happens under the hood in `skchange`.
* Not primarily meant to be used directly by the user.

### Costs -> Savings
As for change scores, all costs can be used to construct a saving. 

```
baseline_cost = MyCost(param=param)
optim_cost = baseline_cost.clone().set_params(param=None)
saving.evaluate([start, end]) = baseline_cost.evaluate([start, end]) - optim_cost.evaluate([start, end])
```

"saving = the cost of the interval with a fixed baseline parameter - the cost of the interval with an optimal parameter"

The baseline parameter represents the normal data behaviour.

### Example 1: Fixed L2 cost
All costs in `skchange` can be configured to evaluate for both a **fixed** and an **optimal** parameter.
```
BaseCost(param: float|np.ndarray|None=None),
```

where

* Optimal: `param=None`
* Fixed: `param: float|np.ndarray`

Recall the single change point data from before.

In [71]:
px.line(single_cpt_df)

In [72]:
from skchange.costs import L2Cost

mean = 0.0
baseline_cost = L2Cost(param=mean)  # fixed mean = 0
baseline_cost.fit(single_cpt_df)
intervals = [[0, 30], [40, 60], [70, 85]]
cost_values = baseline_cost.evaluate(intervals)
plot_interval_costs(single_cpt_df, intervals, cost_values, fixed_mean=mean, optim_mean=False)

* mean=0 is a good fit for [0, 50)
* mean=0 is a poor fit for [50, 100)

### Example 2: Saving based on L2 cost

* Model per interval: A constant mean.
* Output per interval: The squared error for a constant mean model.

In [73]:
from skchange.anomaly_scores import Saving

saving = Saving(baseline_cost=baseline_cost)
saving.fit(single_cpt_df)
savings = saving.evaluate(intervals)
plot_interval_costs(single_cpt_df, intervals, cost_values, fixed_mean=mean, optim_mean=True, cost_name="saving")

/Users/pabloparedes/Documents/darts_sktime_comparing/sktime-tutorial-pydata-global-2024/notebooks/utils.py:180: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



* the optimal mean provides little cost saving over mean=0 for [0, 50)
* the optimal mean provides a large cost saving over mean=0 for [50, 100)

# HVAC system dataset

<img src="img/hvac_system_ventilation.png" alt="img/hvac_system_ventilation.png" width="400"/>

*Heating, ventilation and air conditioning (HVAC) system.*

The dataset:
    
* Two units. We look at one of them.
* Vibration magnitude sensor measurements every 10 minutes.
* 30 days of data.

Data background:
* From the company [Soundsensing](https://www.soundsensing.no/).
* Research project on detecting failing equipment in buildings using vibration and sound sensors.
* Funded by the Research Council of Norway.

In [74]:
from skchange.datasets import load_hvac_system_data

df_hvac = load_hvac_system_data().loc[1]  # only unit 1
df_hvac

,vibration
time,
2023-12-09 04:30:00+00:00,0.004123
2023-12-09 04:40:00+00:00,0.004123
2023-12-09 04:50:00+00:00,0.004123
2023-12-09 05:00:00+00:00,0.004123
2023-12-09 05:10:00+00:00,0.004123
...,...
2024-01-08 03:50:00+00:00,0.004123
2024-01-08 04:00:00+00:00,0.004123
2024-01-08 04:10:00+00:00,0.004123


In [75]:
true_anomaly = pd.Interval(
    pd.Timestamp("2024-01-03 06:00").tz_localize("UTC"),
    pd.Timestamp("2024-01-05 17:00").tz_localize("UTC"),
)

test_start = pd.Timestamp("2024-01-01").tz_localize("UTC")

df_hvac_train = df_hvac.loc[: test_start - pd.Timedelta(seconds=1)]
df_hvac_test = df_hvac.loc[test_start:]

In [76]:
px.line(df_hvac).update_layout(
    yaxis_title="vibration magnitude", showlegend=False
).add_vrect(
    x0=test_start,
    x1=df_hvac.index[-1],
    fillcolor="rgba(0,0,0,0.2)",
    layer="below",
    line_width=0,
    annotation_text="Test set",
    annotation_position="top left",
).add_vrect(
    x0=df_hvac.index[0],
    x1=test_start,
    fillcolor="rgba(0,0,0, 0.05)",
    layer="below",
    line_width=0,
    annotation_text="Train set",
    annotation_position="top left",
).add_vrect(
    x0=true_anomaly.left,
    x1=true_anomaly.right,
    fillcolor="rgba(255,0,0,0.3)",
    line_width=0,
    annotation_text="True anomaly",
    annotation_position="top left",
)

This particular machine has two states:

1. Off: Vibration close to 0
2. On: Vibration more than approximately 0

Task: On each weekday, when does the machine normally turn on and off?

* Used to detect deviations from its regular schedule.
* Useful information to the maintenance staff.

Note: 
* A simple thresholding algorithm could solve the problem in this example.
* Not all cases are as clear-cut!
* Only for demonstration purposes.

### Step 1: Estimate the change points

In [80]:
from skchange.change_detectors import PELT
from skchange.costs import L2Cost

# Standardize the data. Discussed at the end of the example.
std = df_hvac_train.std().iloc[0]
x_train = df_hvac_train / std
x_test = df_hvac_test / std

cost = L2Cost()
change_detector = PELT(cost)
change_detector.fit(x_train)
changepoints = change_detector.predict(x_train)
changepoints

,ilocs
0,296
1,365
2,440
3,509
4,584
5,653
6,728
7,797
8,872
9,941


In [81]:
vib_fig_train = px.line(df_hvac_train).update_layout(
    yaxis_title="vibration magnitude", showlegend=False
)
changepoints["time_locs"] = x_train.index[changepoints["ilocs"]]
add_changepoint_vlines(vib_fig_train, changepoints, "time_locs")

There's also a big difference in variance between the off and on states, so could also use a Gaussian cost.

In [82]:
from skchange.costs import GaussianCost

var_cost = GaussianCost()  # Only line that needs to change
var_change_detector = PELT(var_cost)
var_change_detector.fit(x_train)
var_changepoints = change_detector.predict(x_train)
var_changepoints

,ilocs
0,296
1,365
2,440
3,509
4,584
5,653
6,728
7,797
8,872
9,941


In [84]:
var_changepoints["time_locs"] = x_train.index[changepoints["ilocs"]]
add_changepoint_vlines(vib_fig_train, var_changepoints, "time_locs")

It gives the same result.

We continue with the L2 cost changepoints to detect the on/off times.

### Step 2: Convert the changepoints to on-segments

* Can use the `StatThresholdAnomaliser` in `skchange` for this.
    - Calculate the mean per segment.
    - If the mean in the interval `[stat_lower, stat_upper]` it is a "off-segment".
    - Otherwise, it is an "on-segment".
* Could have also used an anomaly detector like `CAPA` immediately.

In [85]:
from skchange.anomaly_detectors import StatThresholdAnomaliser
from utils import to_time_intervals

change_detector = PELT(L2Cost())
anomaly_detector = StatThresholdAnomaliser(
    change_detector,
    stat=np.mean,
    stat_lower=0.0,
    stat_upper=0.01 / std,  # Since we rescale the data.
)
on_segments = anomaly_detector.fit_predict(x_train)
on_segments = to_time_intervals(on_segments, x_train.index)
on_segments

,ilocs,labels,time_locs
0,"[296, 365)",1,"[2023-12-11 05:50:00+00:00, 2023-12-11 17:20:0..."
1,"[440, 509)",2,"[2023-12-12 05:50:00+00:00, 2023-12-12 17:20:0..."
2,"[584, 653)",3,"[2023-12-13 05:50:00+00:00, 2023-12-13 17:20:0..."
3,"[728, 797)",4,"[2023-12-14 05:50:00+00:00, 2023-12-14 17:20:0..."
4,"[872, 941)",5,"[2023-12-15 05:50:00+00:00, 2023-12-15 17:20:0..."
5,"[1304, 1373)",6,"[2023-12-18 05:50:00+00:00, 2023-12-18 17:20:0..."
6,"[1448, 1517)",7,"[2023-12-19 05:50:00+00:00, 2023-12-19 17:20:0..."
7,"[1592, 1661)",8,"[2023-12-20 05:50:00+00:00, 2023-12-20 17:20:0..."
8,"[1736, 1804)",9,"[2023-12-21 05:50:00+00:00, 2023-12-21 17:20:0..."
9,"[1879, 1948)",10,"[2023-12-22 05:50:00+00:00, 2023-12-22 17:20:0..."


In [86]:
add_segmentation_vrects(vib_fig_train, on_segments, colors=["green"], locs_col="time_locs")

### Step 3: Estimate the weekly schedule

Step 3a: Find the weekday for each on-segment.

In [87]:
on_segments["wday"] = on_segments.apply(
    lambda x: x["time_locs"].left.weekday(),
    axis=1,
)
on_segments

,ilocs,labels,time_locs,wday
0,"[296, 365)",1,"[2023-12-11 05:50:00+00:00, 2023-12-11 17:20:0...",0
1,"[440, 509)",2,"[2023-12-12 05:50:00+00:00, 2023-12-12 17:20:0...",1
2,"[584, 653)",3,"[2023-12-13 05:50:00+00:00, 2023-12-13 17:20:0...",2
3,"[728, 797)",4,"[2023-12-14 05:50:00+00:00, 2023-12-14 17:20:0...",3
4,"[872, 941)",5,"[2023-12-15 05:50:00+00:00, 2023-12-15 17:20:0...",4
5,"[1304, 1373)",6,"[2023-12-18 05:50:00+00:00, 2023-12-18 17:20:0...",0
6,"[1448, 1517)",7,"[2023-12-19 05:50:00+00:00, 2023-12-19 17:20:0...",1
7,"[1592, 1661)",8,"[2023-12-20 05:50:00+00:00, 2023-12-20 17:20:0...",2
8,"[1736, 1804)",9,"[2023-12-21 05:50:00+00:00, 2023-12-21 17:20:0...",3
9,"[1879, 1948)",10,"[2023-12-22 05:50:00+00:00, 2023-12-22 17:20:0...",4


Step 3b: Estimate the on and off times for each weekday.

In [95]:
def estimate_on_off_times(wday_intervals):
    on_times = wday_intervals.array.left
    off_times = wday_intervals.array.right

    on_time = (on_times - on_times.normalize()).mean()
    off_time = (off_times - off_times.normalize()).mean()

    return pd.Series({"on_time": on_time, "off_time": off_time})


schedule = on_segments.groupby("wday")["time_locs"].apply(estimate_on_off_times)
schedule = schedule.unstack()
schedule

,on_time,off_time
wday,,
0,0 days 05:50:00,0 days 17:20:00
1,0 days 05:50:00,0 days 17:20:00
2,0 days 05:50:00,0 days 17:20:00
3,0 days 05:50:00,0 days 17:20:00
4,0 days 05:50:00,0 days 17:20:00


### Step 4: Analyse the test data

In [106]:
vib_fig_test = px.line(df_hvac_test).update_layout(
    yaxis_title="vibration magnitude", showlegend=False
)

test_days = x_test.index.normalize().unique()[:-1]
for day in test_days:
    wday = day.weekday()
    if wday not in schedule.index:
        continue
    on_time = schedule.loc[wday, "on_time"]
    off_time = schedule.loc[wday, "off_time"]
    vib_fig_test.add_vrect(
        x0=day + on_time,
        x1=day + off_time,
        fillcolor="rgba(0,0,255,0.2)",
        line_width=0,
        annotation_text="Expect ON",
        annotation_position="top left",
    )
vib_fig_test.show()

The deviation from the expected schedule can easily be spotted.

**!Alarm!**

### Notes on preprocessing

The default penalties in `skchange` assume that the **within-segment** data has unit variance.

There are currently three options for dealing with this:

1. Estimate the within-segment variance and standardize the data.

    * Common method: Estimate the within-segment variance by `factor*X.diff().var()/2`.
    * Works for data without too much auto-correlation.
2. Tune the penalty to the data directly.
3. A combination of the two.

**Top priority for future development**: More robust and automatic methods for tuning the penalty.


Here we used option 3:

* Rescale the data by the standard deviation of the entire training set.
    - Unless the jumps are enormous, this brings the data to a somewhat common scale.
* Tweak the penalty scale around 1 to get the segmentation right.
    - Penalty scale = 1.0 worked well for this data set. Luck.

# Example: Change in covariance matrix

In [107]:
from scipy.stats import multivariate_normal
import scipy.linalg

p = 10

# Generate a 10x10 covariance matrix
cov_matrix = scipy.linalg.toeplitz(0.9 ** np.arange(p))

# Generate data with two segments (one change point), one with an identity covariance
# matrix and one with the covariance matrix defined above.
values = np.concatenate(
    (
        multivariate_normal.rvs(np.zeros(p), cov_matrix, 100),
        multivariate_normal.rvs(np.zeros(p), np.eye(p), 100),
    )
)
df_cov = pd.DataFrame(values)

# Add a label column
df_cov["label"] = np.concatenate(
    [np.repeat("correlated_segment", 100), np.repeat("independent_segment", 100)]
)

x_train = df_cov.iloc[:, :-1]
plot_multivariate_time_series(x_train)

Not easy to spot the change!

In a scatter plot, however, we can see the change in dependency structure: From a circular to an elliptical shape.

In [108]:
px.scatter(df_cov, x=4, y=5, color="label")

In [110]:
from skchange.costs import MultivariateGaussianCost

cost = MultivariateGaussianCost()
change_detector = PELT(cost=cost, min_segment_length=30)
change_detector.fit(x_train)
change_detector.predict(x_train)

,ilocs
0,100


# Future developement

1. Generalized tuning of hyperparameters across detectors. Penalties/thresholds in particular.
2. Standard preprocessing tools for change and anomaly detection.
3. Plenty more costs, change scores and anomaly scores.

# Credits: Detection notebook

notebook creation: tveten,
<img src="./img/NR-logo_utvidet_stor_rgb.gif" alt="NR logo" style="width: 30%; max-width: 200px;"> 

detection module design: fkiraly, miraep8, alex-jg3, lovkush-a, aiwalter, duydl, katiebuc, johannvk, tveten

# References



* `skchange`: https://github.com/NorskRegnesentral/skchange
* `sktime`: https://www.sktime.net/ 
* `ruptures`:
    - https://centre-borelli.github.io/ruptures-docs/ 
    - C. Truong, L. Oudre, N. Vayatis. Selective review of offline change point\
    detection methods. Signal Processing, 167:107299, 2020.
* `PELT`: Killick, R., Fearnhead, P., & Eckley, I. A. (2012). Optimal detection of \
    changepoints with a linear computational cost. Journal of the American Statistical\
    Association, 107(500), 1590-1598.
* `Seeded binary segmentation`: Kovács, S., Bühlmann, P., Li, H., & Munk, A. (2023).\
    Seeded binary segmentation: a general methodology for fast and optimal changepoint\
    detection. Biometrika, 110(1), 249-256.
* `CAPA` collection: 
    - Fisch, A. T., Eckley, I. A., & Fearnhead, P. (2022). A linear time method\
    for the detection of collective and point anomalies. Statistical Analysis and\
        DataMining: The ASA Data Science Journal, 15(4), 494-508.
    - Fisch, A. T., Eckley, I. A., & Fearnhead, P. (2022). Subset multivariate\
    collective and point anomaly detection. Journal of Computational and Graphical\
    Statistics, 31(2), 574-585.
    - Tveten, M., Eckley, I. A., & Fearnhead, P. (2022). Scalable change-point and\
    anomaly detection in cross-correlated data with an application to condition\
    monitoring. The Annals of Applied Statistics, 16(2), 721-743.